In [ ]:
# Hàm để đặt một lệnh mua
import MetaTrader5 as mt5
from binance.client import Client
from binance.enums import *
import os
# Đặt API key và Secret key
# api_key = "d9OIP56rF3aXhWMq9gmgIyn37uEw0X3D0GzWnm4Mh8fpL9J5bdXmJku4qwh7SHy7"
# api_secret = "HlUyOp8lzMyi3tZ0Q7oWpmrNF58dSbWgEVTApwsIZE6rJSoeS0BNsGZVwy4oEJGW"

# api_key = "9vFZgxlV2iruvGsBIATVTdONbPmN7RdV51Oxwycr8EHfkk2VTxTRLVwdp7Rz2KdM"
api_key = "y1tvZNa7rSq3VMTffUxFOqOWVzrf5Lbu66WlWjR7fw8RJSuNN1Z5JmQ4zBj64I9q"
api_key = 'kMYSAWRigplgKRg16aZONgTs5lRXroWbd33hblLAXDmpT1ddXLXLJJZnbVbE03GY' # Future
# api_secret = "0iWhnDRvGmoOai8xhAhYwdu5bjZ3oCnV5TejQcdOppE96uGnOL2EvdHKfsPB08XJ"
api_secret = "fS5o1d86LUF3IxCtMh3RsHClTaWpDjVA5Ieb5L3y3hCxrg6yoBr7HUq0oIa3PFpV"
api_secret = 'PjsgJVWq5Qy5XHORgXSpzIwQFLxeK15Ja9auiw5TiYCs0vskG93uXO3F3bXxJz18' # Future

def orderSendForex(data):
    # Lấy dữ liệu từ từ điển
    datetime = data['Datetime']
    open_price = data['Open']
    close_price = data['Close']
    volume = data['Volume']
    sma = data['SMA']
    short_ema = data['short_ema']
    long_ema = data['long_ema']
    macd = data['MACD']
    signal_line = data['Signal_Line']
    buy_signal = data['Buy_Signal']
    
    sell_signal = data['Sell_Signal']
    symbol = data['Symbol']
    insertdate = data['Insertdate']
    
    # Khởi tạo client
    client = Client(api_key, api_secret)

    quantity = 5
    leverage = 10  # Đòn bẩy

    # Thay đổi đòn bẩy
    # client.futures_change_leverage(symbol=symbol, leverage=leverage)
    # Thay đổi loại ký quỹ chéo (Cross) sang ký quỹ riêng lẻ (ISOLATED)
    # client.futures_change_margin_type(symbol=symbol, marginType='ISOLATED')

    # Lấy thông tin giá gần nhất của ETHUSDT
    ethusdt_price = client.get_symbol_ticker(symbol=symbol)
    entry_price = ethusdt_price["price"]

    # Chuyển đổi entry_price sang float
    entry_price = float(entry_price)
    
    allowed_precision = 2  # Giả sử độ chính xác tối đa là 6 chữ số thập phân

    # Lấy thông tin về cặp tiền tệ
    info = client.get_symbol_info(symbol)
    # Tìm giới hạn số lượng và giá
    lot_size = next(filter(lambda f: f['filterType'] == 'LOT_SIZE', info['filters']), None)
    price_filter = next(filter(lambda f: f['filterType'] == 'PRICE_FILTER', info['filters']), None)
    # Làm tròn số lượng và giá cho phù hợp với giới hạn
    quantity = round_step_size(quantity, lot_size['stepSize'])
    entry_price = round_step_size(entry_price, price_filter['tickSize'])

    print(quantity)
    print(entry_price)

    order = None

    try:
        if buy_signal == 'True':
            print("Placing a BUY market order.")
            # Đặt Take Profit và Stop Loss (cần tính toán dựa trên chiến lược)
            take_profit = round(entry_price * 1.01, allowed_precision)
            stop_loss = round(entry_price * 0.98, allowed_precision)

            take_profit = round_step_size(entry_price * 1.01, price_filter['tickSize'])
            stop_loss = round_step_size(entry_price * 0.98, price_filter['tickSize'])

            print(take_profit)
            print(stop_loss)
            # Đặt lệnh mua
            buy_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=ORDER_TYPE_MARKET,
                quantity=quantity
            )           
            print("Buy order placed successfully:", buy_order)
            
            print("Placing a TAKE PROFIT market order.")
            tp_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET ,
                timeInForce=TIME_IN_FORCE_GTC,
                quantity=quantity,
                stopPrice=take_profit,
                reduceOnly='true'
            )
            print("Take profit order placed successfully:", tp_order)

            print("Placing a STOP LOSS stop market order.")
            sl_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=FUTURE_ORDER_TYPE_STOP_MARKET,
                quantity=quantity,
                stopPrice=stop_loss,
                reduceOnly='true'
            )
            print("Stop loss order placed successfully:", sl_order)

        elif sell_signal == 'True':
            print("Placing a SELL market order.")
            # Đặt Take Profit và Stop Loss (cần tính toán dựa trên chiến lược)
            take_profit = round(entry_price * 0.99, allowed_precision)
            stop_loss = round(entry_price * 1.02, allowed_precision)

            take_profit = round_step_size(entry_price * 0.98, price_filter['tickSize'])
            stop_loss = round_step_size(entry_price * 1.01, price_filter['tickSize'])

            print(take_profit)
            print(stop_loss)
            
            # Đặt lệnh bán
            sell_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_SELL,
                type=ORDER_TYPE_MARKET,
                quantity=quantity
            )            
            print("Sell order placed successfully:", sell_order)
           
            print("Placing a TAKE PROFIT market order.")
            tp_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=FUTURE_ORDER_TYPE_TAKE_PROFIT_MARKET,
                timeInForce=TIME_IN_FORCE_GTC,
                quantity=quantity,
                stopPrice=take_profit,
                reduceOnly='true'
            )
            print("Take profit order placed successfully:", tp_order)

            print("Placing a STOP LOSS stop market order.")
            sl_order = client.futures_create_order(
                symbol=symbol,
                side=SIDE_BUY,
                type=FUTURE_ORDER_TYPE_STOP_MARKET,
                quantity=quantity,
                stopPrice=stop_loss,
                reduceOnly='true'
            )
            print("Stop loss order placed successfully:", sl_order)

    except Exception as e:
        print(f"An error occurred: {e}")

import math
def round_step_size(quantity, step_size):
    """Làm tròn số lượng theo step size của LOT_SIZE filter."""
    step_size = float(step_size)
    precision = int(round(-math.log(step_size, 10), 0))
    return round(quantity - (quantity % step_size), precision)

In [ ]:
import redis

def entryForex(hash):
    # Xử lý dữ liệu ở đây
    print("Dữ liệu đã được xử lý")

    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)

    # Hash key
    hash_key = hash

    # Đọc dữ liệu từ hash
    data = r.hgetall(hash_key)
    
    if data:
        # Chuyển đổi dữ liệu từ bytes sang chuỗi nếu cần
        data = {key.decode('utf-8'): value.decode('utf-8') for key, value in data.items()}
        # Xử lý dữ liệu
        orderSendForex(data)
        # Xóa hash sau khi xử lý
        r.delete(hash_key)
        print(f"Hash '{hash_key}' đã được xóa khỏi Redis.")
    else:
        print(f"Không tìm thấy dữ liệu cho hash '{hash_key}'.")

In [3]:
from datetime import datetime, timedelta
import schedule
import time

def schedule_entryForex():
    entryForex('OG_CRTO_FT_MACD,MA')

# Lên lịch hàm scan_market để chạy mỗi 15 phút
schedule.every(10).seconds.do(schedule_entryForex)

while True:
    schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
    time.sleep(1)

Dữ liệu đã được xử lý
0.9999
2295.03
Placing a SELL market order.
2249.12
2317.98
An error occurred: APIError(code=-1111): Precision is over the maximum defined for this asset.
Hash 'OG_CRTO_FT_MACD,MA' đã được xóa khỏi Redis.
Dữ liệu đã được xử lý
0.9999
2295.03
Placing a SELL market order.
2249.12
2317.98
An error occurred: APIError(code=-1111): Precision is over the maximum defined for this asset.
Hash 'OG_CRTO_FT_MACD,MA' đã được xóa khỏi Redis.
Dữ liệu đã được xử lý
0.9999
2294.71
Placing a SELL market order.
2248.81
2317.65
An error occurred: APIError(code=-1111): Precision is over the maximum defined for this asset.
Hash 'OG_CRTO_FT_MACD,MA' đã được xóa khỏi Redis.
